In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from tensorflow.keras.layers import Layer, Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from statsmodels.tsa.arima.model import ARIMA

# -----------------------------
# LOAD DATA
# -----------------------------
data = pd.read_csv("multivariate_time_series.csv")

# -----------------------------
# PREPROCESSING
# -----------------------------
scaler = StandardScaler()
scaled_data = pd.DataFrame(
    scaler.fit_transform(data),
    columns=data.columns
)

TypeError: 'type' object is not subscriptable

In [ ]:
def create_sequences(data, target_col, window_size=30):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:i+window_size].values)
        y.append(data.iloc[i+window_size][target_col])
    return np.array(X), np.array(y)

WINDOW_SIZE = 30
X, y = create_sequences(scaled_data, "f1", WINDOW_SIZE)

split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
# -----------------------------
# SELF-ATTENTION LAYER
# -----------------------------
class SelfAttention(Layer):
    def build(self, input_shape):
        self.W = self.add_weight(
            shape=(input_shape[-1], input_shape[-1]),
            initializer="random_normal",
            trainable=True
        )
        self.b = self.add_weight(
            shape=(input_shape[-1],),
            initializer="zeros",
            trainable=True
        )
        self.u = self.add_weight(
            shape=(input_shape[-1],),
            initializer="random_normal",
            trainable=True
        )

    def call(self, inputs):
        score = tf.nn.tanh(tf.tensordot(inputs, self.W, axes=1) + self.b)
        attention_weights = tf.nn.softmax(tf.tensordot(score, self.u, axes=1), axis=1)
        context_vector = tf.reduce_sum(inputs * tf.expand_dims(attention_weights, -1), axis=1)
        return context_vector

In [ ]:
# -----------------------------
# LSTM + ATTENTION MODEL
# -----------------------------
def build_attention_lstm(input_shape):
    inputs = Input(shape=input_shape)
    lstm_out = LSTM(64, return_sequences=True)(inputs)
    attention_out = SelfAttention()(lstm_out)
    output = Dense(1)(attention_out)

    model = Model(inputs, output)
    model.compile(optimizer=Adam(0.001), loss="mse")
    return model

model = build_attention_lstm(X_train.shape[1:])
model.fit(
    X_train, y_train,
    epochs=25,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

In [ ]:
# -----------------------------
# BASELINE LSTM
# -----------------------------
baseline = tf.keras.Sequential([
    LSTM(64, input_shape=X_train.shape[1:]),
    Dense(1)
])
baseline.compile(optimizer="adam", loss="mse")
baseline.fit(X_train, y_train, epochs=25, batch_size=32, verbose=0)

# -----------------------------
# ARIMA BASELINE
# -----------------------------
train_series = data["f1"][:split + WINDOW_SIZE]
test_series = data["f1"][split + WINDOW_SIZE:]

arima = ARIMA(train_series, order=(5,1,0))
arima_model = arima.fit()
arima_preds = arima_model.forecast(steps=len(test_series))


In [ ]:
# -----------------------------
# EVALUATION
# -----------------------------
def evaluate(y_true, y_pred):
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    direction_acc = np.mean(np.sign(np.diff(y_true)) == np.sign(np.diff(y_pred)))
    return rmse, mae, mape, direction_acc

att_preds = model.predict(X_test).flatten()
lstm_preds = baseline.predict(X_test).flatten()

print("\nLSTM + Attention:", evaluate(y_test, att_preds))
print("Simple LSTM:", evaluate(y_test, lstm_preds))